## 8.5 ResNet概述

论文可见于：[论文链接](https://arxiv.org/abs/1512.03385)

ResNet是由微软研究院的 Kaiming He, Xiangyu Zhang, Shaoqing Ren 和 Jian Sun 在 2015 年提出的深度神经网络架构。ResNet斩获当年ImageNet竞赛中分类任务第一名，目标检测第一名；COCO数据集中目标检测第一名，图像分割第一名。它的名字来源于残差网络（Residual Network），也就是 ResNet 架构中的神经网络块是建立在残差模块之上的。

ResNet 是目前依然非常流行的深度神经网络架构，在计算机视觉领域有着广泛的应用，并为之后的深度神经网络架构的提出和改进奠定了基础。截止目前为止，论文引用次数已经超过14万。

ResNet 是为了解决深度神经网络训练时网络退化问题而提出的，所谓的网络退化其实就是深层网络的效果反而比浅层网络更差。ResNet 的提出让我们可以训练出更深的网络，其深度突破了100层，甚至超过了1000层。

### 8.5.1 ResNet基本思想

ResNet 的基本思想是引入残差模块来解决深度神经网络训练时的网络退化问题。

残差模块的思想是将多层神经网络块看成一个整体，然后学习残差，即将输出与输入的差作为模型的预测目标。这样做的好处是可以使梯度流动更加稳定，从而使深度神经网络得以顺利训练。

假设将堆叠的几层神经元称为一个block，那么对于某个block，假设其拟合的函数是 $F(x)$，如果期望潜在映射为 $H(x)$，与其让 $F(x)$ 直接拟合 $H(x)$ ，不如去学习残差 $H(x)-x$，即 $F(x)=H(x)-x$，这样其前向路径就变成了 $F(x)+x$，即 $H(x)=F(x)+x$。其结构如下图所示。

<img src="./images/8-5-1.png" width="50%"></img>

### 8.5.2 ResNet结构

ResNet34的结构与VGG19的对比如下图所示：

<img src="./images/8-5-2.png" width="100%"></img>

其中实现实曲线表示残差连接，虚线表示包含升维。

不同深度的ResNet结构如下图所示：

<img src="./images/8-5-3.png" width="80%"></img>

看上面的结构图好像很复杂，但实际上对照上表就是三部分：

1. 第一部分卷积层+最大池化层；
2. 第二部分是4组不同数量和规格的残差模块；
3. 第三部分平均池化层+全连接层。

### 8.5.3 ResNet代码实现

接下来看一下如何用代码实现相关网络结构。

In [1]:
# 导入必要的库
import torch
import torch.nn as nn

首先定义两种残差模块。

<img src="./images/8-5-4.png" width="60%"></img>

In [2]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [3]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

然后组装ResNet的网络结构。

In [4]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    # 生成网络结构的函数，根据传入的配置，拼接出对应的网络结构
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

最后和VGGNet一样，封装对应模型的函数，直接调用即可。对应layers参数的组合与上文中表格是一致的。

In [5]:
# 封装函数
def resnet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])

def resnet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])

def resnet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

### 8.5.4 小结

* 针对网络退化问题，采用残差结构解决（隔层相连，弱化每层之间的强联系），利用残差结构让网络能够更深、收敛速度更快、优化更容易，同时参数相对之前的模型更少、复杂度更低。
* 对于很深的网络，ResNet使用了更高效的“bottleneck”结构极大程度上降低了参数计算量。
* ResNet深度突破了100层，甚至超过了1000层，对后来的深层神经⽹络设计产⽣了深远影响。